# 케라스로 VGGNet 만들기

이 노트북에서 "매우 깊은" 합성곱 신경망 [VGGNet](https://arxiv.org/pdf/1409.1556.pdf)과 비슷한 모델을 훈련하여 옥스포드 꽃 데이터셋을 17개 카테고리로 분류합니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-3.vggnet_in_keras.ipynb)

#### 동일하게 재현하기 위해 랜덤 시드 값을 지정합니다.

In [1]:
import numpy as np
np.random.seed(42)

#### 라이브러리를 적재합니다.

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard 

#### 데이터를 적재하고 전처리합니다.

이미 `oxflower17.npz` 파일을 다운받았다고 가정합니다. 이 파일이 없다면 `10-2.alexnet_in_keras.ipynb` 노트북을 다시 실행하세요.

In [3]:
# 코랩을 사용할 경우 다음 셀을 실행하세요.
!rm oxflower17*
!wget https://bit.ly/36QytdH -O oxflower17.npz

rm: cannot remove 'oxflower17*': No such file or directory
--2021-08-21 14:52:03--  https://bit.ly/36QytdH
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597859&authkey=AGd0CpvKFkK8GtE [following]
--2021-08-21 14:52:03--  https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597859&authkey=AGd0CpvKFkK8GtE
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://57ucia.bl.files.1drv.com/y4mhsblelfhdnpPhn2g2YfD43TOlVy-Qezv57RBxFqrsny0YYcV8W9AI_Eso_K_JsJeUn6YklxTtBwBOr5GRnWVJPfanGN4SYj8nl9J6q4Q3ITJDU6lb4O0U00FT1rRNYC4WXTpl1p7H1tfqDzpL__EYyq_xei_Lw2wlCqILuslj4h32aCAIn-efBTWSyZGJajhy-6YQZJ

In [4]:
ls -al

total 246516
drwxr-xr-x 1 root root      4096 Aug 21 14:52 ./
drwxr-xr-x 1 root root      4096 Aug 21 14:48 ../
drwxr-xr-x 4 root root      4096 Aug 13 13:34 .config/
-rw-r--r-- 1 root root 252415092 Feb  7  2021 oxflower17.npz
drwxr-xr-x 1 root root      4096 Aug 13 13:35 sample_data/


In [5]:
import numpy as np

data = np.load('oxflower17.npz', allow_pickle=True)
X = data['X']
Y = data['Y']

#### 신경망 모델을 만듭니다.

In [6]:
model = Sequential()

model.add(Conv2D(64, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(128, 3, activation='relu'))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 110, 110, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

#### 모델을 설정합니다.

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 텐서보드를 설정합니다.

In [9]:
tensorbrd = TensorBoard('logs/vggnet')

#### 훈련!

In [10]:
model.fit(X, Y, batch_size=64, epochs=250, verbose=1, validation_split=0.1, shuffle=True, callbacks=[tensorbrd])

Epoch 1/250
20/20 [==============================] - 31s 451ms/step - loss: 2.9429 - accuracy: 0.1642 - val_loss: 31.5923 - val_accuracy: 0.0515
Epoch 2/250
20/20 [==============================] - 7s 337ms/step - loss: 2.7096 - accuracy: 0.2239 - val_loss: 4.7099 - val_accuracy: 0.1029
Epoch 3/250
20/20 [==============================] - 7s 337ms/step - loss: 2.9013 - accuracy: 0.2116 - val_loss: 3.1221 - val_accuracy: 0.1471
Epoch 4/250
20/20 [==============================] - 7s 336ms/step - loss: 2.3329 - accuracy: 0.2525 - val_loss: 3.3452 - val_accuracy: 0.1103
Epoch 5/250
20/20 [==============================] - 7s 336ms/step - loss: 2.1524 - accuracy: 0.2663 - val_loss: 3.3944 - val_accuracy: 0.1103
Epoch 6/250
20/20 [==============================] - 7s 335ms/step - loss: 1.9633 - accuracy: 0.3252 - val_loss: 2.8286 - val_accuracy: 0.1765
Epoch 7/250
20/20 [==============================] - 7s 337ms/step - loss: 1.9303 - accuracy: 0.3309 - val_loss: 2.9324 - val_accuracy: 0.11